## Setup

In [1]:
reset -fs

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron

In [3]:
RANDOM_STATE = 28

## Load Data and Filter

In [4]:
# Comments File
comments = '../Data/CommentsApril2018.csv'

In [5]:
# Read in File
comm = pd.read_csv(comments)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
comm.columns

Index(['approveDate', 'articleID', 'articleWordCount', 'commentBody',
       'commentID', 'commentSequence', 'commentTitle', 'commentType',
       'createDate', 'depth', 'editorsSelection', 'inReplyTo', 'newDesk',
       'parentID', 'parentUserDisplayName', 'permID', 'picURL', 'printPage',
       'recommendations', 'recommendedFlag', 'replyCount', 'reportAbuseFlag',
       'sectionName', 'sharing', 'status', 'timespeople', 'trusted',
       'typeOfMaterial', 'updateDate', 'userDisplayName', 'userID',
       'userLocation', 'userTitle', 'userURL'],
      dtype='object')

In [7]:
# Filter out Unknown & comment commentTypes & Non-NAs
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment") & comm.sectionName.notna()
data = comm.commentBody[filt]
sections = comm.sectionName[filt]

## Train/Test Split and Training

In [8]:
train_data, test_data, train_target, test_target = train_test_split(data, sections, random_state=RANDOM_STATE)

In [9]:
vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
pet_model = Pipeline([('vec', vectorizer),
                    ('pet', Perceptron(random_state=RANDOM_STATE))])
pet_model.fit(train_data, train_target)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s..._jobs=1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))])

## Predictions and Metric Evaluation

In [10]:
pet_model.score(test_data, test_target)

0.6618412572505081

<br>
<br>
<br>

## Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
grid_params = dict(vec__ngram_range=[(1,1), (1,2), (1,3)])

In [13]:
pet_model = Pipeline([('vec', vectorizer),
                      ('clf', Perceptron(n_jobs=-1, random_state=RANDOM_STATE))])

gs = GridSearchCV(estimator=pet_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5,
                 n_jobs=-1)

In [14]:
gs.fit(train_data, train_target)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...jobs=-1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [15]:
gs.best_params_

{'vec__ngram_range': (1, 3)}

In [16]:
gs.best_estimator_

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        s...jobs=-1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))])

In [17]:
gs.score(test_data, test_target)

0.7040305388924694

<br>
<br>
<br>

## With TdidfVectorizor

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer(input='content',
                            analyzer='word',
                            stop_words='english',
                            decode_error='ignore')

pet_model = Pipeline([('vec', vectorizer),
                      ('clf', Perceptron(n_jobs=-1, random_state=RANDOM_STATE))])

In [20]:
pet_model.fit(train_data, train_target)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Pipeline(memory=None,
     steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...jobs=-1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))])

In [21]:
pet_model.score(test_data, test_target)

0.6701700461057954

### GridSearch with TdidfVectorizor

In [22]:
grid_params = dict(vec__ngram_range=[(1,1), (1,2), (1,3)])

In [23]:
pet_model = Pipeline([('vec', vectorizer),
                      ('clf', Perceptron(n_jobs=-1, random_state=RANDOM_STATE))])

gs2 = GridSearchCV(estimator=pet_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5,
                 n_jobs=-1)

In [24]:
gs2.fit(train_data, train_target)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...jobs=-1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [25]:
gs2.best_params_

{'vec__ngram_range': (1, 3)}

In [26]:
gs2.best_estimator_

Pipeline(memory=None,
     steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
   ...jobs=-1, penalty=None, random_state=28,
      shuffle=True, tol=None, verbose=0, warm_start=False))])

In [27]:
gs2.score(test_data, test_target)

0.7141440682167468

<br>
<br>
<br>

## Optimized Model with Classification Report

In [28]:
from sklearn.metrics import classification_report

In [29]:
pet_model = gs2.best_estimator_

In [30]:
predictions = pet_model.predict(test_data)
print(classification_report(test_target, predictions))

                        precision    recall  f1-score   support

                Africa       0.53      0.49      0.51        76
              Americas       0.61      0.50      0.55       232
          Art & Design       0.53      0.50      0.52        80
          Asia Pacific       0.59      0.56      0.58       943
             Australia       0.37      0.43      0.40        23
           Book Review       0.26      0.13      0.18       180
                Canada       0.57      0.71      0.64       133
    College Basketball       0.34      0.56      0.43        18
               Cycling       0.51      0.75      0.61        28
              DealBook       0.50      0.29      0.37        73
                   Eat       0.74      0.69      0.71       119
               Economy       0.56      0.49      0.52       294
                Europe       0.54      0.46      0.50       733
                Family       0.65      0.59      0.62       327
                  Golf       0.53      

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


<br>
<br>
<br>

## Save Model

In [31]:
from joblib import dump, load
dump(pet_model, '../Models/perceptron_small.joblib') 

['../Models/perceptron_small.joblib']

## Save Notebook

In [32]:
import dill
dill.dump_session('../Notebook_Saves/Perceptron_Exploration.db')